# Evaluate Using Bootstrap Metrics

Bootstrap metrics can help us to more thoroughly evaluate a trading strategy, as we will see below.

[In the last notebook](https://www.pybroker.com/en/latest/notebooks/2.%20Backtesting%20a%20Strategy.html), we wrote a trading strategy that we backtested. Here is the implementation again:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

def buy_low(ctx):
    if ctx.long_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] < ctx.low[-2]:
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        ctx.hold_bars = 3
        
def short_high(ctx):
    if ctx.short_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] > ctx.high[-2]:
        ctx.sell_shares = 100
        ctx.hold_bars = 2

And as before, we configure a new [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) instance:

In [2]:
config = StrategyConfig(initial_cash=500_000, bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])
strategy.add_execution(short_high, ['TSLA'])

This time, the ```Strategy``` is configured with a [bootstrap_sample_size](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.bootstrap_sample_size) of ```100``` (the default is ```1_000```). Next, the ```Strategy``` is backtested again, but now with bootstrap metrics enabled by default:

In [3]:
result = strategy.backtest()
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2017-03-01 00:00:00 to 2022-02-28 00:00:00


100% (1259 of 1259) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:01 

Finished backtest: 0:00:03


,name,value
0,trade_count,388.000000
1,initial_market_value,500000.000000
2,end_market_value,693111.870000
3,total_pnl,165740.200000
4,total_return_pct,33.148040
5,total_profit,403511.080000
6,total_loss,-237770.880000
7,total_fees,0.000000
8,max_drawdown,-56721.600000
9,max_drawdown_pct,-7.908429


When looking at ```total_pnl``` above, it appears that we have successfully implemented a profitable trading strategy on our first attempt! But how can we be so sure that those results are repeatable and were not just a fluke? We can gain more confidence in our results by computing metrics using the [boostrap method](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)).

The basic idea behind the bootstrap method is to repeatedly compute a metric on random samples drawn from the backtest results. Then the metric is computed on each random sample and the average is taken. By computing the metric on thousands of random samples, a more robust and accurate estimate of the metric is obtained.

## Confidence Intervals

**PyBroker** uses the bootstrap method to compute [confidence intervals](https://en.wikipedia.org/wiki/Confidence_interval) for the [Profit Factor](https://www.pybroker.com/en/latest/reference/pybroker.eval.html#pybroker.eval.EvalMetrics.profit_factor) and [Sharpe Ratio](https://en.wikipedia.org/wiki/Sharpe_ratio):

In [4]:
result.bootstrap.conf_intervals

lower     upper
name              conf                     
Log Profit Factor 97.5% -0.733064  1.031870
                  95%   -0.592529  0.875702
                  90%   -0.429138  0.699379
Sharpe Ratio      97.5% -0.157705  0.243220
                  95%   -0.125687  0.211407
                  90%   -0.089285  0.173606

Specifically, **PyBroker** used the [bias corrected and accelerated (BCa) bootstrap method](https://blogs.sas.com/content/iml/2017/07/12/bootstrap-bca-interval.html) to compute the confidence intervals above. The returns used for the bootstrap were sampled per-bar rather than per-trade to maximize the information captured by those metrics.

The ``lower`` column holds the lower bound at the given confidence level. The lower bound gives us a "safer" measurement to consider; e.g. we can be ``97.5%`` confident that the Sharpe Ratio is at or above a value of *x*.  

We can see that the lower bounds of the Sharpe Ratio and the (log-transformed) Profit Factor are both negative. This is not a good sign, and shows that our strategy is not reliably profitable!

## Maximum Drawdown

Still, we continue by looking at bootstrap metrics for maximum drawdown: 

In [5]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-299538.19,-36.040592
99%,-225144.03,-28.441203
95%,-172314.35,-22.678177
90%,-145929.62,-19.601042


Shown above are the probabilities that the drawdown will not exceed the computed values, which are given in cash amounts and percentages of portfolio equity. Like the Profit Factor and Sharpe Ratio, these confidence levels were computed using per-bar returns obtained from the backtest's out-of-sample results.

The bootstrapped max drawdown of ```-34.53%``` at a ```99.9%``` confidence level is much worse than the ```-7.9%``` we saw in our original results!

Hopefully, this example gives you a sense of the importance of using randomized tests to analyze (and scrutinize) the performance of your trading strategy.

[The next notebook will go over how to use ranking and position sizing in your trading strategies](https://www.pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html).